# Lecture 7: Working with Claude API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic Claude API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic Claude API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

### Challenge
Take the code we have used in this lecture and see if you can beat my overall score. Let's turn this into a friendly competition for all those tackling this project and compare the results you get :D

Place your submission in the community-contributions folder by creating a folder with your name in it.

### Version History

Below you will find any updates I have made to the notebooks.

**Please note** that some of the code may be different to what you have seen in the lectures as there may be bug fixes or improvements but the general objective of the notebook will remain the same.

| Date | Version | Description of Changes |
|------|---------|------------------------|
| 2025-03-01 | v1.0 | Initial version |



#### Step 1: Import libraries and load the environment variables

In [2]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

#### Step 3: Customize the Prompt

Customer service representative

In [3]:
complaint = "You have over charged me for this product. I demand a refund."

In [4]:
company_policy = """
- We offer refunds only within 14 days of purchase.
- For purchases over $100, a supervisor must approve the refund.
- We prioritize customer satisfaction but aim to minimize refund abuse.

"""

In [8]:
system_message = "You are a customer service agent."

prompt = f"""

We have received a csutomer complaint 20 days ago: "{complaint}"

The company policy is as follows:
{company_policy}

Draft a reply to the customer.

"""
client = anthropic.Client()

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

chat_response = message.content[0].text
display(Markdown(chat_response))

Dear [Customer],

Thank you for reaching out to us regarding your recent purchase. I have reviewed the details of your complaint, and I understand your concern about the charge for the product.

Unfortunately, our company policy states that we only offer refunds within 14 days of the original purchase date. As your purchase was made more than 14 days ago, we are unable to provide a refund at this time.

Additionally, since the purchase amount was over $100, our policy requires that any refund be approved by a supervisor. Given the timeframe of your request, I am not able to authorize a refund without prior approval.

While we strive to prioritize customer satisfaction, we also have measures in place to prevent refund abuse. I understand this may not be the outcome you were hoping for, but I hope you can appreciate the need for these policies to ensure fairness for all our customers.

If you have any other questions or concerns, please do not hesitate to reach out to us. We are committed to providing the best possible service and would be happy to assist you further.

Sincerely,
[Your Name]
Customer Service Representative
[Company Name]

#### Step 4: Rate the response using a reasoning model

In [ ]:
system_message = "You are an expert customer service representative. Evaluate the email response to be sent to a customer complaint."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_response}

Company Policy:
{company_policy}

Rate each category with ONE LINE explanation:

Based on this, decide how the support team should respond. Your analysis must include:
-
-
-
-

"""

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=5000,
    thinking={
        "type": "enabled",
        "budget_tokens": 3000
    },
    temperature=1,
    system=system_message,
    messages=[
        {"role": "user", "content": reasoning_prompt}
    ]
)
reasoning_response = message.content
print(reasoning_response)

[ThinkingBlock(signature='ErUBCkYIBRgCIkDZu2OP+C6U9tOyxtE2Guh92nL070XsINNld3l1NTT/75oqD0LfsrRXGB4jYEHGQAURZCCdeU7TS12nL3knVb3PEgzcIt8Cpow4VBrdCOgaDNt8VxQfXmtTsl+ffyIwzlVkT/HTh9QdyH8bniQA5W/LtBWap6ZIoptlzguydZgDulVL2SyZrLum6XWrEwPdKh2xcYimgQNFAj3un/2r7h4Qntnbq1EDNa9D0nl/CxgC', thinking='I\'ll assess the customer service email response by evaluating several dimensions: appropriateness, clarity, helpfulness, and alignment with both customer service best practices and the stated company policy.\n\nFirst, let\'s analyze the customer complaint:\n"You have over charged me for this product. I demand a refund."\n\nThis is a brief complaint claiming the customer was overcharged and requesting a refund.\n\nNow, let\'s evaluate the response:\n\n1. Tone and Empathy\nThe email begins professionally with acknowledgment of the complaint, but could show more empathy about the overcharging claim. The representative doesn\'t directly address whether the customer was actually overcharged, which is the cor

In [10]:
display(Markdown(reasoning_response[1].text))

# Email Response Evaluation

## Overall Assessment
This response is **inadequate** for addressing the customer's complaint about being overcharged. It focuses on refund policy timing rather than investigating the core claim of incorrect pricing.

## Detailed Rating

**Empathy:** 5/10  
Acknowledges concern but doesn't validate the overcharging claim, which is the customer's primary issue.

**Problem Resolution:** 3/10  
Fails to investigate whether the customer was actually overcharged before denying the refund.

**Clarity:** 7/10  
Clearly explains policy but misapplies it to what might be a billing error, not a standard refund.

**Accuracy:** 4/10  
Makes unverified assumptions about purchase timing without confirming details.

**Customer-Centricity:** 3/10  
References "refund abuse" which appears accusatory when responding to a potential billing error.

## Recommendation

**Decision:** The team should investigate the overcharging claim before deciding on a refund

**Reasoning:** If the customer was genuinely overcharged (billed more than advertised), this represents a billing error that may warrant resolution regardless of the standard refund window.

**Churn Risk:** Very high - customers who believe they've been overcharged and receive a policy rejection without investigation are likely to escalate and ultimately leave.

**Communication Strategy:** 
1. Acknowledge the overcharging claim specifically
2. Request order details to verify the pricing discrepancy
3. If overcharged, resolve regardless of timeframe as it's a company error
4. If not overcharged, explain pricing clearly with evidence

This situation requires fact-finding before policy application, as billing errors fall outside standard refund policies.

# **Your Challenge!!!**

A customer has submitted a complaint. Based on company policy and the situation, the model must select the correct resolution option from a list, and give a 1-line justification.

In [ ]:
complaint = "I ordered a ceramic mug and received a broken one. Please fix this!"

policy = """
- Broken or damaged items must be reported within 7 days of delivery.
- If reported in time, the item is replaced or refunded.
- No action is taken for reports after 7 days unless escalated.
"""

metadata = """
    Reported Days After Delivery: 5
    Item Value: $12
    Customer Tier: Standard
    Past Complaints: 0
"""


Build a customer support agent that could help solve this issue

The agent should choose one of the following options:
- Refund Item
- Replace Item
- Offer Discount
- Apologize (no action)
- Report to supervisor

In [ ]:
system_message = "You are a ..."

prompt = ""

message =